# <font color=3C5055><center><br>Capstone Project<center><br><font color=3C5055>Spark Arrest Classification</font><br><br> Black Belts

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import pyspark
from pyspark.sql.functions import col, isnan, when, count, explode, array, lit
from pyspark.ml.feature import VectorAssembler, StringIndexer

from pyspark.sql.functions import *
from pyspark.sql import functions as F

from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel

## Connect to the Spark server

In [2]:
spark = pyspark.sql.SparkSession.builder.config("spark.executor.memory", "16g").config("spark.driver.memory", "16g").getOrCreate()

## Obtain the Data using Spark

Link to Dataset: https://www.kaggle.com/datasets/salikhussaini49/chicago-crimes

In [3]:
data = spark.read.csv('Chicaco_Crimes.csv',
                     sep=',',
                     inferSchema=True,
                     header=True,
                     multiLine=True)
data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: integer (nullable = true)
 |-- Domestic: integer (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: double (nullable = true)
 |-- Ward: double (nullable = true)
 |-- Community Area: double (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: double (nullable = true)
 |-- Y Coordinate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



### Showing the dataset

In [4]:
data.show()

+---+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+------------------+--------------------+
|_c0|      ID|Case Number|                Date|               Block|IUCR|        Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|         Longitude|            Location|
+---+--------+-----------+--------------------+--------------------+----+--------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+------------------+--------------------+
|  0|10568519|   HZ316563|06/20/2016 11:30:...| 042XX W DIVISION ST|0486|             BATTERY|DOM

### Find the Shape of the dataset

In [5]:
print((data.count(), len(data.columns)))

(1671727, 23)


## Data Preprocessing 



### Removing unnecessary columns 

In [6]:
drop_cols = ['_c0','ID', 'Case Number', 'Location','Description','Latitude','Longitude','X Coordinate','Y Coordinate']

data = data.drop(*drop_cols)

## Data Cleaning

### Finding null values

In [7]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show() 

+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+
|Date|Block|IUCR|Primary Type|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|Updated On|
+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+
|   0|    0|   0|           0|                7490|     0|       0|   0|       1|  50|             1|       0|   0|         0|
+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+



### Drop null values
Because they are lower than 20% of the total dataset, it is better to drop them.

In [8]:
data = data.na.drop()

### Confirm the null values are droped

In [9]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show() 

+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+
|Date|Block|IUCR|Primary Type|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|Updated On|
+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+
|   0|    0|   0|           0|                   0|     0|       0|   0|       0|   0|             0|       0|   0|         0|
+----+-----+----+------------+--------------------+------+--------+----+--------+----+--------------+--------+----+----------+



### Show the shape of data after removing null values and unnecessary columns

In [10]:
print((data.count(), len(data.columns)))

(1664187, 14)


## Descriptive Statistics

### Convert Date and Update-on columns to timestamps and splitting the time into different columns

In [11]:
data = data.withColumn("Date",to_timestamp(col("Date"),format='MM/dd/yyyy hh:mm:ss a'))

In [12]:
data = data.withColumn("Updated On",to_timestamp(col("Updated On"),format='MM/dd/yyyy hh:mm:ss a'))

In [13]:
data = data.withColumn('Day', F.dayofmonth(F.col('date')))
data = data.withColumn('Month', F.month(F.col('date')))
data = data.withColumn('Hour', date_format('date', 'HH'))
data = data.withColumn('Minutes', date_format('date', 'mm'))

data = data.drop('date')

In [14]:
data = data.withColumn('Update_Day', F.dayofmonth(F.col('Updated On')))
data = data.withColumn('Update_Month', F.month(F.col('Updated On')))
data = data.withColumn('Update_Year', F.year(F.col('Updated On')))
data = data.withColumn('Update_Hour', date_format('Updated On', 'HH'))
data = data.withColumn('Update_Minutes', date_format('Updated On', 'mm'))

data = data.drop('Updated On')

### Showing the Schema and Dataset after converting the Dates

In [15]:
data.printSchema()

root
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: integer (nullable = true)
 |-- Domestic: integer (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: double (nullable = true)
 |-- Ward: double (nullable = true)
 |-- Community Area: double (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: string (nullable = true)
 |-- Minutes: string (nullable = true)
 |-- Update_Day: integer (nullable = true)
 |-- Update_Month: integer (nullable = true)
 |-- Update_Year: integer (nullable = true)
 |-- Update_Hour: string (nullable = true)
 |-- Update_Minutes: string (nullable = true)



In [16]:
data.show()

+--------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+---+-----+----+-------+----------+------------+-----------+-----------+--------------+
|               Block|IUCR|        Primary Type|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|Day|Month|Hour|Minutes|Update_Day|Update_Month|Update_Year|Update_Hour|Update_Minutes|
+--------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+---+-----+----+-------+----------+------------+-----------+-----------+--------------+
| 042XX W DIVISION ST|0486|             BATTERY|           APARTMENT|     0|       1|1111|    11.0|37.0|          23.0|     08B|2016| 20|    6|  23|     30|        10|           2|       2018|         15|            50|
| 063XX S STEWART AVE|502P|       OTHER OFFENSE|              STREET|     0|       0| 722|     7.0|20.0|          68.0| 

## Data Processing

### Using Oversampiling to Make the target balanced (Arrest Target) 

In [17]:
major_df = data.filter(col("Arrest") == 0)
minor_df = data.filter(col("Arrest") == 1)
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))

ratio: 4


In [18]:
a = range(ratio)
oversampled_df = minor_df.withColumn("dummy", explode(array([lit(x) for x in a]))).drop('dummy')

In [19]:
oversampled_df.count()

1186488

In [20]:
combined_df = major_df.unionAll(oversampled_df)
combined_df.show()

+--------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+---+-----+----+-------+----------+------------+-----------+-----------+--------------+
|               Block|IUCR|        Primary Type|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|Year|Day|Month|Hour|Minutes|Update_Day|Update_Month|Update_Year|Update_Hour|Update_Minutes|
+--------------------+----+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+----+---+-----+----+-------+----------+------------+-----------+-----------+--------------+
| 042XX W DIVISION ST|0486|             BATTERY|           APARTMENT|     0|       1|1111|    11.0|37.0|          23.0|     08B|2016| 20|    6|  23|     30|        10|           2|       2018|         15|            50|
| 063XX S STEWART AVE|502P|       OTHER OFFENSE|              STREET|     0|       0| 722|     7.0|20.0|          68.0| 

### Confirm the oversampled data

In [21]:
combined_df.groupby('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
|     0|1367565|
|     1|1186488|
+------+-------+



### Do to the fact that we fact we have 35,253 Different Blocks in Chicago, we needed to make them lower for the ML Model to handel them, so we made them 10,000. And make the rest unknown. 

In [22]:
data.select('Block').distinct().count()

35691

In [23]:
n = 10001
most_freq = combined_df.groupBy('Block').count().orderBy('count', ascending=False).take(n - 1)
most_freq = spark.createDataFrame(most_freq).toPandas()
most_freq = most_freq['Block'].tolist()
combined_df = combined_df.withColumn('Block', when(col('Block').isin(most_freq), col('Block')))

In [24]:
combined_df = combined_df.fillna('unknown', subset='Block')

### Showing the final Schema

In [25]:
combined_df.printSchema()

root
 |-- Block: string (nullable = false)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: integer (nullable = true)
 |-- Domestic: integer (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: double (nullable = true)
 |-- Ward: double (nullable = true)
 |-- Community Area: double (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Hour: string (nullable = true)
 |-- Minutes: string (nullable = true)
 |-- Update_Day: integer (nullable = true)
 |-- Update_Month: integer (nullable = true)
 |-- Update_Year: integer (nullable = true)
 |-- Update_Hour: string (nullable = true)
 |-- Update_Minutes: string (nullable = true)



### Using String Indexer to Convert all Categorical Data to Numerical.

In [26]:
cat_cols = ['Block', 'IUCR', 'Primary Type','Location Description','FBI Code','Hour','Minutes','Update_Hour','Update_Minutes']

for c in cat_cols: 
    indexer = StringIndexer(inputCol=c, outputCol=c+'_idx')
    combined_df = indexer.fit(combined_df).transform(combined_df) 
    
final_df = combined_df.drop(*cat_cols)

### Showing the final Dataset

In [27]:
final_df.show()

+------+--------+----+--------+----+--------------+----+---+-----+----------+------------+-----------+---------+--------+----------------+------------------------+------------+--------+-----------+---------------+------------------+
|Arrest|Domestic|Beat|District|Ward|Community Area|Year|Day|Month|Update_Day|Update_Month|Update_Year|Block_idx|IUCR_idx|Primary Type_idx|Location Description_idx|FBI Code_idx|Hour_idx|Minutes_idx|Update_Hour_idx|Update_Minutes_idx|
+------+--------+----+--------+----+--------------+----+---+-----+----------+------------+-----------+---------+--------+----------------+------------------------+------------+--------+-----------+---------------+------------------+
|     0|       1|1111|    11.0|37.0|          23.0|2016| 20|    6|        10|           2|       2018|   2648.0|     0.0|             0.0|                     2.0|         1.0|    14.0|        1.0|            0.0|               0.0|
|     0|       0| 722|     7.0|20.0|          68.0|2016| 21|    6|  

### Spliting the Target from the columns and combining the rest of columns to one feature.

In [28]:
cols = final_df.columns
cols.remove('Arrest')
assembler = VectorAssembler(inputCols=cols, outputCol='features')
final_df = assembler.transform(final_df)

### Final Dataframe

In [29]:
final_df

DataFrame[Arrest: int, Domestic: int, Beat: int, District: double, Ward: double, Community Area: double, Year: int, Day: int, Month: int, Update_Day: int, Update_Month: int, Update_Year: int, Block_idx: double, IUCR_idx: double, Primary Type_idx: double, Location Description_idx: double, FBI Code_idx: double, Hour_idx: double, Minutes_idx: double, Update_Hour_idx: double, Update_Minutes_idx: double, features: vector]

### Split the data into Training (80%) and Testing (20%)

In [30]:
df_data = final_df.select(col('features'), col('Arrest').alias('label'))
df_train, df_test = df_data.randomSplit([0.8, 0.2])

## Model Building

In [31]:
# Import some classifiers and multiclass evaluator
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [32]:
# Decision Tree
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=10001)
model_dt = dt.fit(df_train)

In [33]:
# Random Forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10, maxBins=10001)
model_rf = rf.fit(df_train)

In [34]:
# Logistic Regression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol="label", featuresCol="features")
model_lr = lr.fit(df_train)

In [35]:
# Gradient Boost
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10, maxBins=10001)
model_gbt = gbt.fit(df_train)

In [36]:
# Predicting the results
pred_dt = model_dt.transform(df_test)
pred_rf = model_rf.transform(df_test)
pred_lr = model_lr.transform(df_test)
pred_gbt = model_gbt.transform(df_test)

## Evaluation

In [37]:
# Accuracy Metric
evaluator_A = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# F1 Metric
evaluator_F = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Weighted Precision
evaluator_P = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")

# Weighted Recall
evaluator_R = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Our models
models = [pred_dt, pred_rf, pred_lr, pred_gbt]

# Empty lists 
accuracy = []
F1 = []
precision = []
recall = []

# Fill lists with metric.
for model in models:
    accuracy.append(evaluator_A.evaluate(model))
    F1.append(evaluator_F.evaluate(model))
    precision.append(evaluator_P.evaluate(model))
    recall.append(evaluator_R.evaluate(model))

In [38]:
df_ev = pd.DataFrame(list(zip(accuracy, F1, precision, recall)), 
                     columns = ['Accuracy', 'F1-Score', 'Weighted Precision', 'Weighted Recall'],
                     index = ['Decision Tree', 'Random Forest', 'Logistic Regression', 'Gradient Boost'])

### Showing the Result Dataframe

In [39]:
df_ev

,Accuracy,F1-Score,Weighted Precision,Weighted Recall
Decision Tree,0.731800,0.731713,0.731654,0.731800
Random Forest,0.767599,0.765889,0.769301,0.767599
Logistic Regression,0.534790,0.372690,0.286001,0.534790
Gradient Boost,0.832051,0.831378,0.833031,0.832051


## Using Gridsearch and CrossValidator

In [40]:
grid = ParamGridBuilder().addGrid(gbt.maxIter, [5, 10, 15, 20]).build()
cv = CrossValidator(estimator=gbt, estimatorParamMaps=grid, evaluator=evaluator_A, parallelism=2)
cvModel = cv.fit(df_train)

In [41]:
cvModel.avgMetrics 

[0.8210348291918611, 0.8264598515171896, 0.830008052617219, 0.8326100489626413]

In [42]:
evaluator_A.evaluate(cvModel.transform(df_test))

0.8377324318033794